# src - run_train_nfn_pvtv2_b4_bugged

## Notebook运行提示
- 代码已拆分为多个小单元, 按顺序运行即可在每一步观察输出与中间变量。
- 涉及 `Path(__file__)` 或相对路径的脚本会自动注入 `__file__` 解析逻辑, Notebook 环境下也能引用原项目资源。
- 可在每个单元下追加说明或参数试验记录, 以跟踪核心算法和数据处理步骤。


In [ ]:
from trainer import *
from configure import *

# start here ###################################################################################

# here we reuse the parameters to produce the results of winning kaggle submission.
# at that time, we are in early experiments, training different folds with different
# learning rate, etc ...
#
# later we find that it is sufficient just to use 1e-5 for all folds with 100 epoch,
# then select the model with the est lb metric from the train log.

cfg = deepcopy(default_cfg)
cfg.experiment_name = 'one-stage-nfn-bugged/pvt_v2_b4-decoder3d-01'
cfg.arch = 'pvt_v2_b4'
cfg.comment  = 'xxx'

if 1:  #early training
	for f in [0,1,2,3,4]:
		cfg.fold = f
		cfg.fold_dir = f'{RESULT_DIR}/{cfg.experiment_name}/fold-{cfg.fold}'
		cfg.lr        =  [1e-4, 1e-4, 1e-5, 1e-5, 1e-5][f]
		cfg.num_epoch =  [40, 40, 30, 45, 45][f]

		cfg.resume_from.iteration  = -1
		cfg.resume_from.checkpoint =  None    # cfg.fold_dir + '/checkpoint/00015720.pth'
		run_trainer(cfg)

if 1:  #finetune with smaller rate
	for f in [0,1,2,3,4]:
		cfg.fold = f
		cfg.lr        =  1e-5
		cfg.num_epoch =  100

		cfg.resume_from.iteration  = -1
		cfg.resume_from.checkpoint =  [
			cfg.fold_dir + '/checkpoint/00010850.pth',
			cfg.fold_dir + '/checkpoint/00012544.pth',
			cfg.fold_dir + '/checkpoint/00009925.pth',
			cfg.fold_dir + '/checkpoint/00015720.pth',
			cfg.fold_dir + '/checkpoint/00015520.pth',
		][f]
		run_trainer(cfg)